In [5]:
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

import logging
import pathlib, os

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

In [22]:
#### Download scifact.zip dataset and unzip the dataset
# dataset = "scifact"
# url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
# out_dir = os.path.join(pathlib.Path('__file__').parent.absolute(), "datasets")
# data_path = util.download_and_unzip(url, out_dir)

#### Provide the data_path where learncorpus dataset is located
data_path = "./datasets/learncorpus"


In [30]:
#### Provide the data_path where scifact has been downloaded and unzipped
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

2024-12-12 12:52:28 - Loading Corpus...


100%|██████████| 10/10 [00:00<00:00, 9929.70it/s]

2024-12-12 12:52:28 - Loaded 10 TEST Documents.
2024-12-12 12:52:28 - Doc Example: {'text': '# Connect functions to Azure services using bindings (programming-language-csharp)\nWhen you create a function, language-specific trigger code is added in your project from a set of trigger templates. If you want to connect your function to other services by using input or output bindings, you have to add specific binding definitions in your function. To learn more about bindings, see [Azure Functions triggers and bindings concepts](functions-triggers-bindings).\n## Local development\nWhen you develop functions locally, you need to update the function code to add bindings. For languages that use function.json, Visual Studio Code provides tooling to add bindings to a function.\n### Manually add bindings based on examples\nWhen adding a binding to an existing function, you need to add binding-specific attributes to the function definition in code.\nThe following example shows the function definit

In [31]:
qrels

{'0': {'f07eec0a-b6ae-4368-35a2-e026ef58a10b-066': 1},
 '1': {'cf374970-651e-b078-cf54-7a1117d11405-001': 1},
 '2': {'cf374970-651e-b078-cf54-7a1117d11405-001': 1},
 '3': {'cf374970-651e-b078-cf54-7a1117d11405-001': 1},
 '4': {'cf374970-651e-b078-cf54-7a1117d11405-001': 1},
 '5': {'cf374970-651e-b078-cf54-7a1117d11405-002': 1},
 '6': {'cf374970-651e-b078-cf54-7a1117d11405-002': 1},
 '7': {'cf374970-651e-b078-cf54-7a1117d11405-002': 1},
 '8': {'cf374970-651e-b078-cf54-7a1117d11405-002': 1},
 '9': {'cf374970-651e-b078-cf54-7a1117d11405-002': 1},
 '10': {'cf374970-651e-b078-cf54-7a1117d11405-003': 1},
 '11': {'cf374970-651e-b078-cf54-7a1117d11405-003': 1},
 '12': {'cf374970-651e-b078-cf54-7a1117d11405-003': 1},
 '13': {'cf374970-651e-b078-cf54-7a1117d11405-003': 1},
 '14': {'cf374970-651e-b078-cf54-7a1117d11405-003': 1},
 '15': {'cf374970-651e-b078-cf54-7a1117d11405-004': 1},
 '16': {'cf374970-651e-b078-cf54-7a1117d11405-004': 1},
 '17': {'cf374970-651e-b078-cf54-7a1117d11405-004': 1},
 '

In [9]:
corpus
first_corpus = {'31715818': corpus['31715818'], '14717500': corpus['14717500']}
print(first_corpus)

KeyError: '31715818'

In [8]:
queries
first_queries = {list(queries.keys())[0]: list(queries.values())[0], list(queries.keys())[1]: list(queries.values())[1]}
print(first_queries)

{'0': 'How do I add input or output bindings to an Azure function?', '1': 'What is the difference between using an isolated process and in-process for Azure function definitions?'}


In [25]:
queries
for key, value in queries.items():
    print(f"Query ID: {key}, Query: {value}")


Query ID: 0, Query: How do I add input or output bindings to an Azure function?
Query ID: 1, Query: What is the difference between using an isolated process and in-process for Azure function definitions?
Query ID: 2, Query: Can you explain what a MultiResponse object is in Azure Functions?
Query ID: 3, Query: What changes should I make if I'm using ASP.NET Core integration in my Azure functions?
Query ID: 4, Query: Where can I find examples of different binding types for Azure Functions?


In [37]:
#### Load the SBERT model and retrieve using cosine-similarity
# model = DRES(models.SentenceBERT("msmarco-distilbert-base-tas-b"), batch_size=16)
# retriever = EvaluateRetrieval(model, score_function="cos_sim") # or "cos_sim" for cosine similarity
# results = retriever.retrieve(first_corpus, first_queries)
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from urllib.parse import parse_qs, urlparse
import json
import requests


def convert_v2_to_v1(res: dict) -> list:
    # The Knowledge Service response schema has been changed from v1 to v2
    # To align with the format in Learn Copilot Journey DB, we take v1 format as the inner contract
    lst = res.get("items", [])
    ret = [
        {
            "itemId": item["id"],
            "score": item["rerankerScore"],
            "metadata": {
                "content": item["content"],
                "lastUpdated": item["lastModifiedDateTime"],
                "url": item["contentUrl"],
                "title": item["title"],
            },
        }
        for item in lst
    ]
    return ret

def fetch_knowledge_service_response(
    token: str,
    query: str,
    endpoint: str,
    category: str,
    filter: str,
    scope: str,
    top: int,
    threshold: float,
) -> list[dict]:
    # Asserts
    assert category in {
        "document",
        "training",
        "evaluation",
    }, f"Invalid category {category}"
    assert top > 0, f"Invalid top {top} when invoke knowledge service, must be > 0."
    assert (
        0 <= threshold <= 1
    ), f"Invalid threshold {threshold} when invoke knowledge service, must be [0, 1]."

    # if endpoint.startswith("https://learn.microsoft.com"):
    #     assert (
    #         category != "evaluation"
    #     ), "Evaluation category is not supported in public KS."

    # Fetch response
    endpoint_url = endpoint.replace("{category}", category)
    endpoint_url_parsed = urlparse(endpoint_url)

    url_params = parse_qs(endpoint_url_parsed.query)
    endpoint_url_parsed = endpoint_url_parsed._replace(params="", query="", fragment="")

    request_params = {
        "url": endpoint_url_parsed.geturl(),
        "json": {
            "input": query,
            "filter": filter,
        },
        "headers": {"Authorization": f"Bearer {token}"},
        "params": {
            **url_params,
            "top": top,
            "scorethreshold": threshold,
        },
    }

    use_api_v2 = category == "document"
    if use_api_v2:
        request_params["params"]["api-version"] = "v2"
    # if scope is not None or scope.strip("\n ") != "":
    #     request_params["params"]["scope"] = scope

    # response = post_request(request_params=request_params)
    response = requests.post(
        request_params["url"],
        json=request_params["json"],
        headers=request_params["headers"],
        params=request_params["params"],
    )
    response.raise_for_status()

    return convert_v2_to_v1(response.json()) if use_api_v2 else response.json()
    #return response.json()

def retrieve_from_ks(
    token: str,
    query: str,
    endpoint: str,
    ks_category: str = "document",
    ks_filter: str = None,  # type: ignore
    ks_scope: str = None,  # type: ignore
    ks_top: int = 10,
    ks_threshold: float = 0.8,
) -> object:

    # try:
    #     ks_const = json.loads(ks_source)

    #     if isinstance(ks_const, list):
    #         if validate_ks_const(ks_const):
    #             return ks_const
    # except AssertionError as e:
    #     raise e
    # except Exception:
    #     pass

    return fetch_knowledge_service_response(
        token,
        query,
        endpoint,
        ks_category,
        ks_filter,
        ks_scope,
        ks_top,
        threshold=ks_threshold,
    )

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "api://5405974b-a0ac-4de0-80e0-9efe337ea291/.default"
)

results = {qid: {} for qid in queries.keys()}

for key, value in queries.items():
    ks_result = retrieve_from_ks(token_provider(), value, "https://learn.microsoft.com/api/knowledge/document/relevantitems", "document", "", None, 10, 0.1)
    results[key] = {item['itemId']: item['score'] for item in ks_result}
    #results.append({key: {item['itemId']: item['score'] for item in ks_result}})

2024-12-12 13:27:38 - No environment configuration found.
2024-12-12 13:27:38 - ManagedIdentityCredential will use IMDS
2024-12-12 13:27:38 - Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.17.1 Python/3.12.4 (Windows-11-10.0.26100-SP0)'
No body was attached to the request
2024-12-12 13:27:41 - DefaultAzureCredential acquired a token from AzureCliCredential


In [28]:
results

{'0': {'f07eec0a-b6ae-4368-35a2-e026ef58a10b-066': 3.6816015243530273,
  'f07eec0a-b6ae-4368-35a2-e026ef58a10b-085': 3.6433188915252686,
  'f07eec0a-b6ae-4368-35a2-e026ef58a10b-045': 3.6192150115966797,
  'cf374970-651e-b078-cf54-7a1117d11405-015': 3.4426894187927246,
  'cf374970-651e-b078-cf54-7a1117d11405-031': 3.4072425365448},
 '1': {'a73dc11f-a9bc-a240-6220-c86d525721d2-001': 2.8280398845672607,
  'a73dc11f-a9bc-a240-6220-c86d525721d2-003': 2.7032668590545654,
  'a73dc11f-a9bc-a240-6220-c86d525721d2-002': 2.6965317726135254,
  'a73dc11f-a9bc-a240-6220-c86d525721d2-004': 2.6929872035980225,
  '8af6b738-7017-145b-dd8c-b0b688f3d15e-012': 2.570695400238037},
 '2': {'03192d8e-54da-76e7-375a-8eea4bbb78c6-024': 2.4480490684509277,
  'cbd9c203-06d8-4413-6da5-c7451a3011c0-028': 2.4423775672912598,
  'c34e0088-f672-23af-d32b-e743f86ea934-001': 2.4409596920013428,
  '902511ac-31d3-ae46-74ae-bf83c1aeb12d-007': 2.257610559463501,
  '9b4697b4-b0de-574b-761b-57e92ead0cd8-007': 2.238557815551758}

In [38]:
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
#### Evaluate your model with NDCG@k, MAP@K, Recall@K and Precision@K  where k = [1,3,5,10,100,1000] 

model = DRES(models.SentenceBERT("msmarco-distilbert-base-tas-b"), batch_size=16)
retriever = EvaluateRetrieval(model, score_function="cos_sim") # or "cos_sim" for cosine similarity
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

2024-12-12 13:29:14 - Use pytorch device_name: cpu
2024-12-12 13:29:14 - Load pretrained SentenceTransformer: msmarco-distilbert-base-tas-b
2024-12-12 13:29:18 - For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
2024-12-12 13:29:18 - 

2024-12-12 13:29:18 - NDCG@1: 0.0164
2024-12-12 13:29:18 - NDCG@3: 0.0164
2024-12-12 13:29:18 - NDCG@5: 0.0164
2024-12-12 13:29:18 - NDCG@10: 0.0270
2024-12-12 13:29:18 - NDCG@100: 0.0270
2024-12-12 13:29:18 - NDCG@1000: 0.0270
2024-12-12 13:29:18 - 

2024-12-12 13:29:18 - MAP@1: 0.0164
2024-12-12 13:29:18 - MAP@3: 0.0164
2024-12-12 13:29:18 - MAP@5: 0.0164
2024-12-12 13:29:18 - MAP@10: 0.0208
2024-12-12 13:29:18 - MAP@100: 0.0208
2024-12-12 13:29:18 - MAP@1000: 0.0208
2024-12-12 13:29:18 - 

2024-12-12 13:29:18 - Recall@1: 0.0164
2024-12-12 13:29:18 - Recall@3: 0.0164
2024-12-12 13:29:18 - Recall@5: 0.0164
2024-12-12 13:29:18 - Recall@10: 0.0492
2024-12-12 13:29:18 